## Plotting Candle Sticks

Welcome to this ipython notebook for plotting candle sticks. We are going to use numpy, quandl and plotly in order to acquire data and plot out candle sticks

In [453]:
import talib
import numpy as np
import quandl
import plotly.plotly as py
import plotly.graph_objs as go
from datetime import datetime
import plotly
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plot


Here we can specify the specific stock info that we want. Search the quandl free api calls here(https://www.quandl.com/search?query=) Currently I have a call bellow that gets bitcoin prices on a day resolution. We get the Open, Close, High, Low, Volume(BTC), Volume(Currency) and weighted price values for each day. These are stored in a pandas data frame and can be accesed i.e. stock_val['open']. All of the data has a date value associated with it.

In [ ]:
stock_val=quandl.get("BCHARTS/BITFINEXUSD")

In [ ]:
stock_val.describe()

Next we are going to make a nice candle stick plot of the data. In order to do this you have to setup an account at plotly(https://plot.ly). After you setup an account make sure to confiure your account according to these directions(https://plot.ly/python/getting-started/). Once this is done you should be able to run the bellow and get out a simple candle stick chart. Check out other charts here(https://plot.ly/python/candlestick-charts/)

In [441]:
trace = go.Candlestick(x=stock_val.index,
                       open=stock_val.Open,
                       high=stock_val.High,
                       low=stock_val.Low,
                       close=stock_val.Close)
data = [trace]
py.iplot(data, filename='simple_candlestick')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~lthul/0 or inside your plot.ly account where it is named 'simple_candlestick'


In [442]:
RB = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.Open)
US = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.High - stock_val.Open)
LS = 100.0 * (stock_val.Close - stock_val.Open) / (stock_val.Close - stock_val.Low)
data = np.zeros((RB.size,3))
data[:,0] = RB.values
data[:,1] = US.values
data[:,2] = LS.values
inputdata = torch.Tensor(data)

In [443]:
D_in, H, D_out = 3, 2, 1


In [444]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [505]:
input_size, hidden_size, output_size = 100, 128, 1

In [516]:
def getInputTensor(x, index):
    inputTensor = x[0, index:(index+100)].unsqueeze(0)
    return inputTensor


In [507]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input.unsqueeze(0), hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1,self.hidden_size))



In [508]:
rnn = RNN(input_size, hidden_size, output_size)
learning_rate = 0.0005
criterion = nn.MSELoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)


In [509]:
x = Variable(torch.Tensor(RB.values)).unsqueeze(0)
y = Variable(torch.Tensor(RB[2:len(RB)].values)).unsqueeze(0)


In [510]:
def train(input_tensor, output_tensor):
    loss = 0
    hidden = rnn.initHidden()

    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)
        loss += criterion(output, output_tensor[i])
        optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     for p in rnn.parameters():
#         p.data.add_(-learning_rate, p.grad.data)

    return output, loss.data[0] / input_tensor.size()[0]

import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [511]:
rnn = RNN(input_size, hidden_size, output_size)

n_iters = 1000
print_every = 100
plot_every = 100
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(0, n_iters + 1):
    output, loss = train(x[0, (iter):(iter+100)].unsqueeze(0), y[0, iter].unsqueeze(0))
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 0s (0 0%) 218.3377
0m 0s (100 10%) 21.6333
0m 0s (200 20%) 59.8745
0m 0s (300 30%) 61.3775
0m 0s (400 40%) 5.8834
0m 0s (500 50%) 7.1114
0m 0s (600 60%) 20.1230
0m 0s (700 70%) 5.4103
0m 0s (800 80%) 0.3697
0m 0s (900 90%) 0.5287
0m 0s (1000 100%) 0.1403


In [521]:
for b in range(1, 1000):
    input_tensor = getInputTensor(x, b)
    for i in range(input_tensor.size()[0]):
            output, hidden = rnn(input_tensor[i], hidden)
            print('%d' % output)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [519]:
hidden

Variable containing:

Columns 0 to 9 
 0.6937  0.3390 -0.7889  0.4215 -0.0121  1.0903 -0.2892  0.5155 -0.3974  0.7126

Columns 10 to 19 
-0.1741  1.0575 -0.8649 -0.1002  1.3356  0.3620  0.4566 -1.2366 -0.1316 -1.3285

Columns 20 to 29 
 0.3961  0.1117 -1.5145  0.7261 -0.4745 -0.3512 -0.3169  0.0241  0.2603 -0.2415

Columns 30 to 39 
-0.2525  0.7452  0.1150 -0.7694 -0.9978  0.5902  0.7874  0.8196 -0.5640  0.4232

Columns 40 to 49 
-0.3148  0.2652 -0.9801  0.0969  0.2035 -2.2882  0.6050  0.2795  0.0910 -0.2284

Columns 50 to 59 
-0.1591 -0.3336 -0.1005 -0.0218 -1.4211 -2.0512  0.9724 -0.0266 -1.7655  0.9743

Columns 60 to 69 
 0.4673 -0.3022  0.4396 -0.2671  1.1371  0.4935  0.5110 -0.6589  1.2384 -1.5553

Columns 70 to 79 
-0.6510  1.4336  0.2970 -1.4757 -0.4706  0.1081 -2.2605 -0.7190  0.1625 -1.5578

Columns 80 to 89 
 0.5041 -0.4743  0.0670  0.6094 -0.7039  1.0076 -1.8557 -0.0829 -0.0552  0.8701

Columns 90 to 99 
-0.2374 -1.3443 -0.6991 -0.2503  0.3955 -0.9638 -0.3709  0.2561  0.8030